<a href="https://colab.research.google.com/github/kmjohnson3/Intro-to-MRI/blob/master/NoteBooks/Selective_RF_Excitation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatially selective excitation
This module will explore slice selection in which we aim to excite a slice (2D imaging) or slab (3D imaging).

First we will need to run some code to import libraries and define a Bloch solver. This time the Bloch solver has some code to make it faster but slightly less accurate. 

In [1]:
# This is comment, Python will ignore this line

# Import libraries (load libraries which provide some functions)
%matplotlib inline
import numpy as np # array library
import math
import cmath
from scipy import interpolate
import numba

# For interactive plotting
from ipywidgets import interact, interactive, FloatSlider, ToggleButton
from IPython.display import clear_output, display, HTML

# for plotting modified style for better visualization
import matplotlib.pyplot as plt 
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 4
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['legend.fontsize'] = 16

@numba.jit(nopython=True)
def bloch_solver( B, time, freq=0, T1=2000, T2=2000, GAM=42.58e6*2*math.pi):
  # This is simple Rk4 solution to the Bloch Equations.
  #
  # Inputs:
  #   B(array)    -- Magentic Field [N x 3] (T)
  #   time(array) -- Time of each point in waveforms (s)
  #   freq        -- Frequency [Hz]
  #   T1          -- Longitudinal relaxation times (s)
  #   T2          -- Transverse relaxation times (s)
  #   M0          -- Initial state of magnetization (not equilibrium magnetization)
  # Outputs:
  #   MOutput -- Magnetization for each position in time

  # Convert frequency to rads/s
  act_freq = 2*math.pi*freq

  #Convert to rotion rates (gamma*B)
  Bx =  GAM*B[:,0]
  By =  GAM*B[:,1]
  Bz =  GAM*B[:,2] + act_freq 
  
  # Double the resolution using linear interpolation (this is faster than splines)
  Bx2 = np.zeros( 2*len(Bz)+2)
  By2 = np.zeros( 2*len(Bz)+2)
  Bz2 = np.zeros( 2*len(Bz)+2)
  Bx2[:-4:2] = Bx[:-1]
  By2[:-4:2] = By[:-1]
  Bz2[:-4:2] = Bz[:-1]

  # Temp
  Bx2[1:-3:2] = 0.5*Bx[:-1] + 0.5*Bx[1:] 
  By2[1:-3:2] = 0.5*By[:-1] + 0.5*By[1:]
  Bz2[1:-3:2] = 0.5*Bz[:-1] + 0.5*Bz[1:]
    
  #Initialize
  Mag = np.array([[0.0],[0.0],[1.0]])

  # Output storage
  MOutput = np.zeros_like(B)
  MOutput = np.expand_dims(MOutput,-1)

  #Runge-Kutta PDE Solution
  dt = time[2] - time[1]
  for count, t1 in enumerate(time):
   
    m1 = Mag
  
    bx = Bx2[count*2]
    by = By2[count*2]
    bz = Bz2[count*2]
    rhs = np.array([[  -1/T2,   bz,  -by],
                  [  -bz   ,-1/T2, bx],
                  [   by   , -bx, -1/T1]])

    k1 = np.dot(rhs, m1) + np.array([[0.0],[0.0],[1.0/T1]])
  
    t2 = t1 + dt/2
    bx = Bx2[count*2+1]
    by = By2[count*2+1]
    bz = Bz2[count*2+1]
    m2 = Mag + k1*dt/2
    k2 = np.dot(np.array([[  -1/T2,   bz,  -by],
                  [  -bz   ,-1/T2, bx],
                  [   by   ,  -bx, -1/T1]]), m2) + np.array([[0.0],[0.0],[1.0/T1]])


    t3 = t1 + dt/2
    bx = Bx2[count*2+1]
    by = By2[count*2+1]
    bz = Bz2[count*2+1]
    m3 = Mag + k2*dt/2
    k3 = np.dot(np.array([[  -1/T2,   bz,  -by],
                  [  -bz   ,-1/T2, bx],
                  [   by   ,  -bx, -1/T1]]), m3) + np.array([[0.0],[0.0],[1.0/T1]])

    t4 = t1 + dt
    bx = Bx2[count*2+2]
    by = By2[count*2+2]
    bz = Bz2[count*2+2]
    m4 = Mag + k3*dt
    k4 = np.dot(np.array([[  -1/T2,   bz,  -by],
                  [  -bz   ,-1/T2, bx],
                  [   by   ,  -bx, -1/T1]]), m4) + np.array([[0.0],[0.0],[1.0/T1]])

    # Runge-Kutta averages the above terms  
    Mag = Mag + dt/6*(k1 + 2*k2 + 2*k3 + k4);

    # Save to an array
    MOutput[count,:]= Mag
    
  return MOutput

# Sinc Pulses 

Not all pulses in MRI are Sinc pulses but we will consider pulses that are. Our pulses will have several paramaters:

* **TBW** [unitless]: The time bandwidth product. This is effectively how many of the sinc lobes we include. More lobes means higher selectivity
* **T** [s]: The time length of the RF pulse, this will control the total time the RF pulse takes. It will set *BW* [Hz], the bandwidth of the pulse in Hz
* **Window function** : This is a function that rolls off the ends of the sinc so that there is a smoother transition when cutting off lobes. For this exercise I am using a hamming window but other options exist. 

The below code generated the pulse envelope. In this code, $B_1$ will be aligned in $x$ such that it rotates the magnetization into the $y$ direction. The sinc can also be modulated by a frequency to excite at a different center frequency.


In [2]:
def generate_sinc(T, TBW=4, window=True, dt=4e-6, GAM=42.58e6, flip=10, freq=0):
  
  # Number of points in waveform
  Nt = int(T/dt)

  # Time normalized to the time bandwidth product
  t = np.linspace(-TBW,TBW, Nt)

  # Get the pulse shape
  B1 = np.sinc(t)

  # To deal with the truncation we can apply a window function to taper the RF profile
  if window:
    B1 *= np.hamming(Nt)

  # Normalize to the flip angle
  B1 = B1 * (flip/360) / (GAM*np.sum(B1*dt))

  # Get actual time
  time = dt*np.arange(Nt)

  # Convert to complex with frequency
  B1 = B1*np.exp(2j*math.pi*time*freq)

  return time, B1

def simulate_rf(time, B1):
  B = np.zeros( (len(B1),3))
  B[:,0] = np.real(B1)
  B[:,1] = np.imag(B1)
  
  Mout = bloch_solver( B, time, T1=2000, T2=2000, GAM=42.58e6*2*math.pi)

  return Mout

def plot_rf(T, TBW, flip, freq, window):
  # Create Sinc
  time, B1 = generate_sinc(T/1e3, TBW, flip=flip, window=window, freq=freq)

  # Simulate
  Mout = simulate_rf( time, B1)

  plt.figure(figsize=(12,6))
  plt.subplot(121)
  plt.plot(1e3*time,np.real(B1),label='$B_x$')
  plt.plot(1e3*time,np.imag(B1),label='$B_y$')
  plt.xlabel('Time [ms]')
  plt.ylabel('$B_1$ [T]')
  plt.legend()


  plt.subplot(122)
  plt.plot(1e3*time,Mout[:,2], label=r'$M_z$')
  plt.plot(1e3*time,Mout[:,0], label=r'$M_x$')
  plt.plot(1e3*time,Mout[:,1], label=r'$M_y$')
  plt.xlabel('Time [ms]')
  plt.ylabel('Magnetization [a.u.]')
  
  plt.legend()
  plt.show()

# Sinc scaling parameters without gradients

Below is a simulation using a standard Bloch simulator. The paramaters are set to maintain a constant flip angle for on-resonant spins. Try the following purtibations, first thinking what the affect might be on the peak $B_1$ which is often limited on systems.
* Change the flip angle
* Change the the TBW and T
* Sweep the frequency, does the flip angle change? Is this different for a short and long pulse? 


In [3]:
w = interactive(plot_rf, 
                TBW=FloatSlider(min=1, max=12, step=1, value=2, description='TBW '),
                T=FloatSlider(min=0.5, max=10, step=0.5, value=2, description='T [ms]'),
                flip=FloatSlider(min=1, max=90, step=1, value=20, description='Flip [deg.]'),
                freq=FloatSlider(min=-2000, max=2000, step=100, value=0, description='RF Freq [Hz]'),
                window=ToggleButton(value=True,description='Toggle Window'))
display(w)

interactive(children=(FloatSlider(value=2.0, description='T [ms]', max=10.0, min=0.5, step=0.5), FloatSlider(v…

# Adding a slice select gradient
 Now we will add a slice select gradient. In this code, the time of the pulse scales with the $TBW$ by:
\begin{equation}
T = TBW*0.25 \times 10^{-3}
\end{equation}

This means that the bandwidth of the pulse ($BW$) is fixed to:
\begin{equation}
BW=\frac{TBW}{T} = \frac{TBW}{TBW*0.25 \times 10^{-3} [s]}= 4000 [Hz]
\end{equation}

This makes seeing many of the effects much easier. Some questions to consider:

* What is the effect of changing the center frequency? Does it depend on the gradient strength?
* How does changing the gradient amplitude affect the slice thickness?
* Does toggling the window function alter the slice profile?
* What might be the practical need for the rephasing gradient?
* Does the profile look the same for a 90 degree flip angle as a 15 degree? [the small tip angle aproximation will be violated and the responsse will not be a Fourier transform]


In [4]:

def simulate_rf_g(time, G, z, B1, T1, T2):
  B = np.zeros( (len(B1),3))
  B[:,0] = np.real(B1)
  B[:,1] = np.imag(B1)
  B[:,2] = G*z
    
  Mout = bloch_solver( B, time)

  return Mout

def generate_sinc_and_grad( Gsel=1e-3, TBW=4, flip=10, window=True, freq=0, rephase=True):

  T = 0.25e-3*TBW

  # Generate RF
  time, B1 = generate_sinc( T, TBW, flip=flip, window=window, freq=freq)

  # Get delta time
  dt = time[1] - time[0]

  # Gradient for slice select (T/m)
  gselect = Gsel*np.ones_like(time)
  
  if rephase:
    # Generate rephaser 
    Grephase = 20e-3 # amplitude of rephaser
    T_rephase = (Gsel * T * 0.5) / Grephase # Area / Gradient strength
    Nrephase = int(np.ceil( T_rephase / dt )) # number of points
    grephase = -Grephase*np.ones((Nrephase,)) #actual
    grephase = -grephase*0.5*np.sum(gselect)/np.sum(grephase)
    
    # Pad with zeros
    pad = np.zeros((20,))
    G = np.concatenate( (pad, gselect, grephase, pad))
    B1 = np.concatenate( (pad, B1, 0*grephase, pad))
    time = dt*np.arange(len(B1))
  else:
    # Pad with zeros
    pad = np.zeros((20,))
    G = np.concatenate( (pad, gselect, pad))
    B1 = np.concatenate( (pad, B1, pad))
    time = dt*np.arange(len(B1))

  return time, B1, G

def plot_rf_g(TBW, flip, Gsel, freq, window, rephase):
  
  # Essentially ignore T1/T2
  T1=1000
  T2=1000

  # Convert to si units
  Gsel=Gsel/1e3 # mT/m to T/m

  # Create Sinc
  time, B1, G = generate_sinc_and_grad( Gsel=Gsel, TBW=TBW, flip=flip, window=window, freq=freq, rephase=rephase)

  ## Simulate
  zsim = np.linspace(-0.05,0.05,501) # Z values to simulate
  Mall = []
  for z in zsim:
    Mout = simulate_rf_g(time, G, z, B1, T1, T2)
    Mall.append(Mout)
  Mall = np.stack(Mall,axis=0)

  # Plots
  fig=plt.figure(figsize=(12,8))
  
  # Plot gradients
  plt.subplot(221)
  plt.plot(1e3*time, 1e3*G, color='b')
  plt.ylim([-25, 25])
  plt.xlabel('Time [ms]')
  plt.ylabel('G [mT/m]', color='b')
  
  # Plot B1
  plt.subplot(223)
  plt.plot(1e3*time,np.real(B1),label='$B_x$')
  plt.plot(1e3*time,np.imag(B1),label='$B_y$')
  plt.xlabel('Time [ms]')
  plt.ylabel('$B_1$ [T]')
  plt.legend()
  plt.ylim([-1.2*np.max(np.abs(B1)), 1.2*np.max(np.abs(B1))])
  plt.xlabel('Time [ms]')

  # Plot of Mz
  plt.subplot(222)
  plt.plot(zsim*1e3,Mall[:,-1,2],label='$M_z$')
  plt.xlabel('Position [mm]')
  plt.ylabel('M [a.u.]')

  # Plot of Mxy
  plt.subplot(224)
  plt.plot(zsim*1e3,Mall[:,-1,1],label='$M_y$')
  plt.plot(zsim*1e3,Mall[:,-1,0],label='$M_x$')
  plt.xlabel('Position [mm]')
  plt.ylabel('M [a.u.]')
  plt.legend()
  
  plt.tight_layout(pad=0.4, w_pad=4.0, h_pad=1.0)
  plt.show()



In [5]:

w = interactive(plot_rf_g, 
                TBW=FloatSlider(min=1, max=12, step=1, value=6, description='TBW',continuous_update=False),
                flip=FloatSlider(min=1, max=90, step=1, value=10, description='Flip [deg.]',continuous_update=False),
                freq=FloatSlider(min=-5000, max=5000, step=100, value=0, description='Freq [Hz]',continuous_update=False),
                Gsel=FloatSlider(min=3, max=20, step=1, value=10,description='Gsel [mT/m]', continuous_update=False),
                window=ToggleButton(value=True,description='Toggle Window',continuous_update=False),
                rephase=ToggleButton(value=True,description='Toggle Rephaser',continuous_update=False),
                )
display(w)

interactive(children=(FloatSlider(value=6.0, continuous_update=False, description='TBW', max=12.0, min=1.0, st…